# XAUUSD

In [35]:
import quandl
import pandas as pd
import numpy as np

# plotting modules
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import os

import warnings
import logging

# scikit-learn modules
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_array


# User defined modules
from column_transformers.technical_indicators import MacdSignal, StochRsiSignal, StochasticRsi
from column_transformers.alpha_factors import BaseStrategy
from column_transformers.dates import DateDummy

from itertools import product
from operator import itemgetter

In [9]:
xau_ratios = [
    "WGC/GOLD_DAILY_USD"
     #"WGC/GOLD_DAILY_EUR",
#     "WGC/GOLD_DAILY_TRY",
#     "WGC/GOLD_DAILY_JPY",
    #"WGC/GOLD_DAILY_GBP",
#     "WGC/GOLD_DAILY_CAD",
#     "WGC/GOLD_DAILY_CHF",
#     "WGC/GOLD_DAILY_VND",
#     "WGC/GOLD_DAILY_KRW",
#     "WGC/GOLD_DAILY_RUB",
#     "WGC/GOLD_DAILY_AUD",
]

economic_indc = []

### Quandl data terms

Anyone seeking to use this code must first apply for an account with [Quandl](https://www.quandl.com) in order to receive an valid authetitciation key.

In [10]:
DIR_NAME = os.path.abspath(os.path.join(os.getcwd(), '..'))
FILEPATH = os.path.join(DIR_NAME, "auth.txt")

with open(FILEPATH, "r") as f:
    authtoken = f.read();

# Retrieve Data

In [11]:
xau_df_dict = {}

for ratio in tqdm(xau_ratios):
    name = ratio.lower().replace("/", "_")

    # get the ratio dataframe
    df = quandl.get(ratio, authtoken=authtoken, start_date = "1979-01-01")
    df.columns = ["price"]
    
    # check for missing business days 
    if pd.infer_freq(df.index) != "B":
        logging.warn("Datetime frequency is not Business Days")
    
    xau_df_dict[name] = df

100%|██████████| 1/1 [00:03<00:00,  3.06s/it]


## Volatility

In [12]:
annualization_factor = 252.
window_size = [5, 10, 20, 60, 120]

for ratio, df in tqdm(xau_df_dict.items()):
    start_date, end_date = df.index[0], df.index[-1]
    full_range = pd.date_range(start_date, end_date, freq = "B")
    
    if not np.array_equal(df.index, pd.date_range(start_date, end_date, freq="B")):
        logging.warning("\n{} is missing business days".format(ratio))

    for window in window_size:
        df['{}d_market_vol'.format(window)] = np.sqrt(
            (annualization_factor/window) * df['price'].rolling(window).var(ddof=0))

100%|██████████| 1/1 [00:00<00:00,  1.84it/s]


## Quandl Features

In [13]:
features = [
    "FRED/T10Y2Y",
    "RATEINF/INFLATION_USA",
]

In [14]:
for ratio, df in xau_df_dict.items():
    for feature in features:
        col_name = feature.lower().replace('/', '_')
        
        # get quandl features. `end_date` is set to df.index[-1] to match the price data
        data = quandl.get(feature, authtoken=authtoken, start_date = "1979-01-01", end_date = df.index[-1])
        start_date, end_date = data.index[0], data.index[-1]

        # Some features contain missing data. To best simulate how the data would be ingested
        # realtime, the current value is forward filled. This achieved by resampling.
        if not np.array_equal(data.index, pd.date_range(start_date, end_date, freq="B")):
            logging.warning("\n\t{} is missing business days".format(feature))
                    
        df[col_name] = data
    
    df[:] = df.ffill()

	FRED/T10Y2Y is missing business days
	RATEINF/INFLATION_USA is missing business days


## Technical indicator features

In [17]:
import talib

technical_indicators= {
#     "MACD" : ("macd", "macdsignal", "macdhist"),
#     "STOCHRSI" : ("fastk", "fastd"),
    "MOM" : ("real"),
    "APO" : ('real'),
    "RSI" : ('real')
}

for ratio, df in xau_df_dict.items():
    
    # talib requires market price data. starting price of $1 is taken
    # since absolute values are not important (preprocess scaling)
    price = df['price'].values

    for indicator, indicator_type in technical_indicators.items():
        # Return the result for each indicator
        if indicator == 'STOCHRSI':
            result = getattr(talib, indicator)(price, fastd_matype = 8)
        else:
            result = getattr(talib, indicator)(price)

        if isinstance(result, np.ndarray):
            df[indicator.lower()] = result
        else:
            for f, r in zip(indicator_type, result):
                if f == indicator.lower():
                    df["{}".format(indicator.lower())] = r
                else:
                    df["{}_{}".format(indicator.lower(), f)] = r

## Strategies

In [47]:
class StochasticRsiStrategy(BaseStrategy):
    
    # ======================================================================
    # Constants
    # ======================================================================
    
    """
    Define the indices of the price series and the to be insered indicators
    See Documentation for more information.
    """
    PRICE = 0 
    FASTK, FASTD = 11, 12
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs) 


    def _price_indicator(self, X, timeperiod, fastk, fastd):
        ind = StochasticRsi(self.PRICE, timeperiod, fastk, fastd)
        
        return ind.fit_transform(X)
        
    def _long_signal(self, price_indicator, long_entry, long_exit):
        
        # Use np.insert if shift is greater than 1
        signal_entry = price_indicator[:, self.FASTK] > long_entry
        signal_hold = price_indicator[:, self.FASTK] > long_exit
        
        # Define the long signal
        long = signal_entry | signal_hold
        
        return long[:-1] * 1
    
    
    def _short_signal(self, price_indicator, short_entry, short_exit):
        
        #Use np.insert if shift is greater than 1
        signal_entry = price_indicator[:, self.FASTK] < short_entry
        signal_hold = price_indicator[:, self.FASTK] <= short_exit
        
        # Define the long signal
        short = signal_entry | signal_hold
        
        return short[:-1] * -1    

    def x(self, X):
        X = check_array(X)
        return X[:, self.PRICE]


class MacdStrategy(BaseStrategy):
    
    # ======================================================================
    # Constants
    # ======================================================================
    
    """
    Define the indices of the price series and the to be insered indicators
    See Documentation for more information.
    """
    PRICE = 0 
    MACD, MACD_SIGNAL, MACD_HIST = 14, 15, 16
    
    # ======================================================================
    # Constants
    # ======================================================================
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def _price_indicator(self, X, fast_period, slow_period, signal_period):
                
        real = X[:, self.PRICE]
        
        macd_statistics = talib.MACD(
            real,
            fastperiod = fast_period,
            slowperiod = slow_period,
            signalperiod = signal_period
        )   
        
        return np.c_[X, np.array(macd_statistics).T]
    
    def _long_signal(self, price_indicator):
        long = price_indicator[:,self.MACD] > price_indicator[:,self.MACD_SIGNAL]
        
        return long[:-1] * 1
    
    
    def _short_signal(self, price_indicator):
        short = price_indicator[:, self.MACD] < price_indicator[:, self.MACD_SIGNAL]
       
        return short[:-1] * -1
    
class StochasticRsi(BaseEstimator, TransformerMixin):
    
    # ======================================================================
    # Constructors
    # ======================================================================
    
    def __init__(self, price_index, timeperiod=14, k=3, d=3):
        self.price_index = price_index
        self.timeperiod = timeperiod
        self.k = k
        self.d = d
       
    # ======================================================================
    # Indicator methods
    # ======================================================================
    
    def _rsi(self, X):
        real = X[:, self.price_index]
        return talib.RSI(real, timeperiod = self.timeperiod)
    
    
    def _stoch_rsi(self, X):
        rsi = self._rsi(X) 
        
        max_rsi = talib.MAX(rsi, timeperiod = self.timeperiod)
        min_rsi = talib.MIN(rsi, timeperiod = self.timeperiod)
        
        fastk = talib.SMA(
            real = ((rsi - min_rsi) / (max_rsi - min_rsi)) * 100,
            timeperiod = self.k
        )
        fastd = talib.SMA(real=fastk, timeperiod=self.d) 
        
        return np.c_[fastk, fastd]

    
    # ======================================================================
    # Public methods
    # ======================================================================
    
    
    def fit(self, X, y=None):
        return self

    
    def transform(self, X, y=None):
        stoch_rsi = self._stoch_rsi(X)
        
        return np.c_[X, stoch_rsi]

### Plotting function to complete

In [ ]:
# longs = X_rsi.index[X_rsi['long'] == 1]
# shorts = X_rsi.index[X_rsi['short'] == -1]

# # start date positions of new long/short positions
# long_indices_or_sections = np.arange(longs.size)[longs.to_series().diff() > pd.Timedelta('3D')]
# short_indices_or_sections = np.arange(shorts.size)[shorts.to_series().diff() > pd.Timedelta('3D')]

# long_date_regions = np.split(longs, long_indices_or_sections)
# short_date_regions = np.split(shorts, short_indices_or_sections)

In [ ]:
# sns.set(rc={'figure.figsize':(16, 10)})
# fig, axes = plt.subplots(nrows=3, ncols=1)

# df.loc['1990', 'alpha_perf'].plot(ax = axes[0])
# df.loc['1990', ['macd', 'macd_macdsignal']].plot(ax=axes[1])
# df.loc['1990', 'gold_perf'].plot(ax = axes[2])

# for l_period, s_period in zip(long_date_regions, short_date_regions):
#     for ax in axes:
#         ax.axvline(l_period[0], color='green', linewidth=1)
#         ax.axvline(s_period[0], color='green', linewidth=1)
                                             
#         ax.axvline(l_period[-1], color='red', linewidth=1)
#         ax.axvline(s_period[-1], color='red', linewidth=1)

#         ax.axvspan(l_period[0], l_period[-1], alpha = 0.1, color = 'green')
#         ax.axvspan(s_period[0], s_period[-1], alpha = 0.1, color = 'red')

## Data preprocessing 

In [24]:
from split._split import TrainValidateTest

data = xau_df_dict['wgc_gold_daily_usd'].copy()
data['target'] = data['price'].shift(-1).pct_change()
#data.dropna(inplace = True)

X = data.drop(['target'], axis = 1)
y = (data['target'] > 0).astype(int)
    
split = TrainValidateTest(0.7, 0.15, 0.15)
X_train, X_valid,  X_test, y_train, y_valid,  y_test = split.transform(X, y)

X_train.head()

,price,5d_market_vol,10d_market_vol,20d_market_vol,60d_market_vol,120d_market_vol,fred_t10y2y,rateinf_inflation_usa,mom,apo,rsi
Date,,,,,,,,,,,
1979-01-01,226.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-02,226.80,NaN,NaN,NaN,NaN,NaN,-0.82,NaN,NaN,NaN,NaN
1979-01-03,218.60,NaN,NaN,NaN,NaN,NaN,-0.82,NaN,NaN,NaN,NaN
1979-01-04,223.15,NaN,NaN,NaN,NaN,NaN,-0.77,NaN,NaN,NaN,NaN
1979-01-05,225.50,21.053213,NaN,NaN,NaN,NaN,-0.77,NaN,NaN,NaN,NaN


In [55]:
stoch_params = dict(timeperiod = range(10,20, 2), 
                    fastk = range(2, 5), 
                    fastd = range(2, 5), 
                    ob_region = range(45, 60, 5), 
                    os_region = range(0, 15, 5))

macd_params = dict(fast_period = range(5, 10),
                   slow_period = range(20, 40),
                   signal_period = range(5, 25, 5))

In [56]:
preprocess_pipeline = Pipeline([
     ('stoch_ud_signal', StochasticRsiStrategy(**stoch_params)),
     ('macd_ud_signal', MacdStrategy(**macd_params))
     #('date', DateDummy('weekday_name', 'month_name')),
     #('vol_diff', VolatilityDiff()),
     #('scalar', StandardScaler())
])

X_train_trans, X_valid_trans= (
    preprocess_pipeline.fit_transform(X_train),
    preprocess_pipeline.transform(X_valid)
)

100%|██████████| 400/400 [00:00<00:00, 1124.17it/s]


In [58]:
X_train_trans.shape

(7435, 18)

## Model Selection

In [ ]:
classifiers = [
    SVC(gamma=2, C=1),
    LogisticRegression(),
    RandomForestClassifier(criterion='entropy', oob_score=True, n_jobs=-1, random_state= 0),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GradientBoostingClassifier(n_estimators=100),
]

In [ ]:
results = {}

for clf in tqdm(classifiers, unit='Model') :
    clf.fit(X_train_trans, y_train)
    
    name = str(clf).split('(')[0]
    
    results[name] = {
        "train_score" : clf.score(X_train_trans, y_train),  
        "valid_score" : clf.score(X_valid_trans, y_valid)
    }      
    
results

## Model Evaluation

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

tscv = TimeSeriesSplit(n_splits = 2)
clf =  RandomForestClassifier(criterion='entropy', oob_score=True, n_jobs=-1, random_state= 0)
rf_param_grid = {
    'max_depth': [25, 30],
    'min_samples_leaf': [10, 15, 16],
    'min_samples_split': [2, 5],
    'n_estimators': [50, 100, 150]
}

search = GridSearchCV(estimator=clf, cv=tscv, param_grid=rf_param_grid)
search.fit(X_train_trans, y_train)

In [ ]:
search.score(X_train_trans, y_train), search.score(X_valid_trans, y_valid)
search.best_params_

In [ ]:
train_score

In [ ]:
n_days = X_train.shape[0]
n_features = X_train.shape[1]

clf_parameters = {
    'criterion': 'entropy',
    'min_samples_leaf': 15,
    'max_depth' : 25,
     'min_samples_split': 8,
    'oob_score': True,
    'n_jobs': -1,
    'random_state': 0}

n_trees_l = [75, 100, 150, 300]

In [ ]:
train_score = []
valid_score = []
oob_score = []
feature_importances = []

for n_trees in tqdm(n_trees_l, desc='Training Models', unit='Model'):
    
    clf = RandomForestClassifier(**search.best_params_)
    clf.fit(X_train_trans, y_train)
    
    train_score.append(clf.score(X_train_trans, y_train))
    valid_score.append(clf.score(X_valid_trans, y_valid))
    
#     oob_score.append(clf.oob_score_)
#     feature_importances.append(clf.feature_importances_)

In [ ]:
train_score, valid_score


In [ ]:
def plot(xs, ys, labels, title='', x_label='', y_label=''):
    for x, y, label in zip(xs, ys, labels):
        plt.ylim((0.3, 0.9))
        plt.plot(x, y, label=label)
        
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)

    plt.legend(bbox_to_anchor=(1.04, 1), borderaxespad=0)
    plt.show()

plot([n_trees_l]*3,
    [train_score, valid_score, oob_score],
    ['train', 'validation', 'oob'],
    'Random Forrest Accuracy',
    'Number of Trees')

In [ ]:
prob_array=[-1,1]
alpha_score = clf.predict_proba(X_train_trans).dot(np.array(prob_array))

# calculate daily returns
alpha_return = alpha_score * data.loc[X_train.index, 'target'].shift(-1) 

xau_usd_return = data.loc[X_train.index, 'target']

# calculate cumulative performance
alpha_perf = 100000 * ((1 + alpha_return).cumprod())
xau_usd_per = 100000 * ((1 + xau_usd_return).cumprod())

#alpha_return.plot()
alpha_sharpe = (np.sqrt(252.) * alpha_return.mean()) / alpha_return.std() 
xau_usd_sharpe = (np.sqrt(252.) * xau_usd_return.mean()) / xau_usd_return.std() 

alpha_perf.plot()
xau_usd_per.plot()

xau_usd_return.mean() / alpha_return.mean() 
sns.set(rc={'figure.figsize':(15, 8)})
plt.legend(labels = ['Model', 'Actual Gold/USD'])


print(alpha_sharpe, xau_usd_sharpe)
alpha_return.std() / xau_usd_return.std()

alpha_perf[-2] ** (252 / alpha_perf.size) - 1 

In [ ]:
from sklearn.model_selection import learning_curve
train_sizes=np.linspace(.1, 1.0, 5)
estimator = RandomForestClassifier(**search.best_params_)
cv = TimeSeriesSplit(n_splits = 5)


In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

plot_learning_curve(estimator, "lol", X, y, n_jobs =4, cv =cv, train_sizes=train_sizes)

In [ ]:
#learning_curve(
        #estimator, X, y, cv=cv, n_jobs=4, train_sizes=train_sizes)
    
cv